# LangChain demo

## Links
1. [samwit - colab - YT LangChain - Agents.ipynb, React & Serpai ](https://colab.research.google.com/drive/1QpvUHQzpHPvlMgBElwd5NJQ6qpryVeWE?usp=sharing)
1. [samwit - github youtube samples](https://github.com/samwit/langchain-tutorials/tree/main)
1. [Building Custom Tools and Agents with LangChain (gpt-3.5-turbo)](https://www.youtube.com/watch?v=biS8G8x8DdA): [samwit - colab - YT LangChain Custom Tools & Agents.ipynb](https://colab.research.google.com/drive/1FYsa3x3PzziL57EHEIuIqa5rkCAxCbin?usp=sharing) uses chat-conversational-react-description

## Demo use case

### Tools
1. IncidentTool. Simple text
1. KnowledgeBaseTool: Vectorstore 
1. NewRelicTool: PandasAI
1. OrgSearchTool: Team, people, rules of engagement

### Flow
1. Get inc
1. Describe steps for user to create/trigger
    1. Narrative
    1. Bullet point flow
1. Describe what happened
    1. Narrative
    1. Bullet point flow
1. Team to direct inc to
    1. Team/person
    1. How to engage (i.e., email, Absa Snow with configs)
1. Extraction

In [1]:
import os
import json

# Load setting from Json outside of project.
f = open('../../env/ai.json')
settingsJson = json.load(f)
del f

for key in settingsJson:
    os.environ[key] = settingsJson[key]
    
del settingsJson
    
# # OR manually set them
# os.environ['REQUESTS_CA_BUNDLE'] = '../../env/ZCert.pem'
# os.environ['HUGGING_FACE_API_KEY'] = 'Get here: https://huggingface.co/settings/tokens'
# os.environ['OPENAI_API_KEY'] = 'Get here: https://platform.openai.com/account/api-keys'
# os.environ["SERPAPI_API_KEY"] = 'serpapi KEY, Get here: https://serpapi.com/manage-api-key'


In [2]:
#!pip install langchain unstructured openai chromadb Cython tiktoken pypdf


# Old code
        #!pip install unstructured
        # I had issues here so ran the following to make PC trust PIP
        # pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org pip setuptools
        # pip config set global.trusted-host "pypi.org files.pythonhosted.org pypi.python.org"

        #!pip install markdown
        #!pip install urllib3==1.25.11

        #!pip install pypdf
        
        #!pip install chromadb
        # For chromadb you have to have C++ builders intalled see link below: "I have faced similar issue on Windows OS, while doing..."
        #   https://stackoverflow.com/questions/73969269/error-could-not-build-wheels-for-hnswlib-which-is-required-to-install-pyprojec

        #!pip install pydantic-settings

In [3]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

# Tools 
from langchain.agents import Tool
from langchain.tools import BaseTool

In [4]:
llm = OpenAI(temperature=0)


In [5]:
# # Load incidents into vector store

# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import Chroma
# from langchain.text_splitter import CharacterTextSplitter

# with open('data/incident-store.txt') as f:
#     txt1 = f.read()

# splitter1 = CharacterTextSplitter(
#     separator='split_me', 
#     chunk_size = 1000,
#     chunk_overlap  = 200,
#     length_function = len,
#     is_separator_regex = False,
# )

# incidentPages1 = splitter1.create_documents([txt1])
# incidentEmbeddings1 = OpenAIEmbeddings()
# incidentDb = Chroma.from_documents(incidentPages1, incidentEmbeddings1) 

# # incidentDb = Chroma.from_documents(incidentPages1, incidentEmbeddings1, persist_directory='data') 
# # incidentDb.persist()

# # result = incidentDb.similarity_search("Bob Jones", k=1) 
# # print(result)
# print(len(incidentPages1))
# print(len(incidentPages1))

# result = incidentDb.similarity_search("INC123", k=1) 
# print(result)
# x = 1

## Create custom tool

# Simple Text Tool
Returns simple text

In [6]:

# Simple Text Tool backed 
incidents = []
with open('./data/incident-store.txt') as f:
    incidents = json.load(f)

def incident_search(incident_number: str):        
    for inc in incidents:
        #print(f'**************** {inc}')
        if (inc['incident_number'] == incident_number ):
            return inc
        
    return f'Could not find incident {incident_number}'

incident_search_tool = Tool(
    name='Incident tool', 
    func= incident_search,    
    description="""
        Useful for looking up incidents, pass in an incident number to lookup an incident.        
        I an incident has a closed_date = 'N/A' then it's probaly still Open
        """
)

#print(len(incidents))
# x = incident_search('INC456')
# print(x)

# Vectorstore 
Vectorstore
https://python.langchain.com/docs/integrations/toolkits/vectorstore \
\
Markdown \
Try this later on \
https://python.langchain.com/docs/modules/data_connection/document_loaders/markdown \
https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/markdown_header_metadata 



In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/knowledge-base.pdf")
internalKbPages = loader.load_and_split()

embeddings = OpenAIEmbeddings()
knowledgeBaseDb = Chroma.from_documents(internalKbPages, embeddings) #, collection_name='internal-knowledge-base-1')

# Vectorstore Knowledgebase Query Tool
The query hits the vector store

In [8]:
def internal_knowledgebase_search(query: str):
    result = knowledgeBaseDb.similarity_search(query, k=1)        
    return  result

internal_knowledgebase_tool = Tool(
    name='Internal Knowledgebase tool', 
    func= internal_knowledgebase_search,    
    description="""
        Useful for querying about our internal services, teams and how to solve customer problems.
        The internal knowledge base has steps on how to solve customers problems, how our software systems work and which people and teams to reach out to. 
        Ask it for questions like 'What should I do?', 'What are the causes of this?', 'What do I do next?'
        """
)

# Setup agent

In [9]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.agents import initialize_agent

tools = [incident_search_tool, internal_knowledgebase_tool]

memory = ConversationBufferWindowMemory(memory_key='chat_history', k=3, return_messages=True)
turbo_llm = ChatOpenAI(temperature=0.9, model_name='gpt-3.5-turbo')

"""
Questions to ask...

Question: My forward prices are not correct.
Question: What did the user do to create the latest incident?
Question: How do I troubleshoot a user who's application keeps crashing when they click detach?
Question: My FX Prices are incorrect

True facts: latest inc is OTP

#######################
Where it works:
Question: What tools do you have availbale to yourself?
Question: Please give me an update on incident number INC789?
Question: Please give me an update on incident number INC456?
Question: Please give me an update on incident number INC123?
Question: What is the latest incident?
Question: What are the common causes of software crashes?

If an incident's state is 'Open' then you must do a query the internal knowledge base in your investigations

#######################
Where it doesn't work:

Question: Please help me troubleshoot INC789

If the question is a reference to an incident first look at the incident and then search the internal knowledge base for an answer otherwise just look in the internal knowledge base.
"""

# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

question = """
You are a support engineer who is helping troubleshoot software issues. 
You responses need to be specific and factful, they can only contain information that you get back from the tools and observations.
You must include the source of your answers.
If you don't know the answer then say so and don't make up anything. 
Explain each step in how you got to your answer. 

#####################################################################################################
Question: "Please provide information about the recent incident related to INC789.

#####################################################################################################

"""
answer = conversational_agent(question)
print(answer)



> Entering new AgentExecutor chain...
{
    "action": "Incident tool",
    "action_input": "INC789"
}
Observation: {'split_me': '|', 'incident_number': 'INC789', 'short_description': 'UI Application keeps crashing', 'long_description': 'Bob Jones at MegaCorpB has the app crashing when he clicks on detach. ', 'created_date': '2023-07-21', 'closed_date': 'N/A', 'state': 'Open', 'troubleshooting_conversation': [{'comment': '2023-09-21 09:30: @SupportMember2 sent email to customer and waiting for feedback from the customer about setting up a call to try and see what is going on.'}, {'comment': '2023-09-21 10:30: @SupportMember2 had call with the customer, there were some windows updates pending. They are going to do these updates after work today and feedback'}], 'resolution_notes': 'Not resolved'}
Thought:{
    "action": "Final Answer",
    "action_input": "The recent incident related to INC789 is still open. The short description of the incident is 'UI Application keeps crashing'. Bob 

In [10]:
answer = conversational_agent("""
    Now, please check the internal knowledgebase for any troubleshooting steps or runbooks related to this incident.
    """)
print(answer)



> Entering new AgentExecutor chain...
{
    "action": "Internal Knowledgebase tool",
    "action_input": "Troubleshooting steps for incident INC789"
}
Observation: [Document(page_content='Runbooks  \nRates Error Mapping Runbook  \nThis section describes how to troubleshoot issues with rates. If users ar e not getting ticking rates this is \nwhere you must come to troubleshoot.  \nMappings incorrect  \nIf mapping are incorrect for a given pair/tenor then you will not be able to subscribe to them. Users will \nnot get ticking rates.  \n \nFX UI Runbooks  \nUser does not get OTP sent to them on login  \nWe have seen this happen when the customer service provider does not forward the OTP onto the \ncustomer. If this happens it is out of our control as we are dependent on the network service providers \nfor this to happen.  \nWhat should I do?  \n1. Reach out to the Communications Delivery Team to see if there are issues with this network \nprovider in sending messages  \n2. If there are 

In [11]:
answer = conversational_agent("""
    Now, please check the internal knowledgebase for which team would be best suited to solve this problem?
    """)
print(answer)



> Entering new AgentExecutor chain...
{
    "action": "Internal Knowledgebase tool",
    "action_input": "Which team is best suited to solve the problem with app crashing when clicking 'detach'?"
}
Observation: [Document(page_content='Runbooks  \nRates Error Mapping Runbook  \nThis section describes how to troubleshoot issues with rates. If users ar e not getting ticking rates this is \nwhere you must come to troubleshoot.  \nMappings incorrect  \nIf mapping are incorrect for a given pair/tenor then you will not be able to subscribe to them. Users will \nnot get ticking rates.  \n \nFX UI Runbooks  \nUser does not get OTP sent to them on login  \nWe have seen this happen when the customer service provider does not forward the OTP onto the \ncustomer. If this happens it is out of our control as we are dependent on the network service providers \nfor this to happen.  \nWhat should I do?  \n1. Reach out to the Communications Delivery Team to see if there are issues with this network \np

In [12]:
answer = conversational_agent("""
    Whos is the support staff who is working with this incident?
    """)
print(answer)



> Entering new AgentExecutor chain...
{
    "action": "Incident tool",
    "action_input": "INC789"
}
Observation: {'split_me': '|', 'incident_number': 'INC789', 'short_description': 'UI Application keeps crashing', 'long_description': 'Bob Jones at MegaCorpB has the app crashing when he clicks on detach. ', 'created_date': '2023-07-21', 'closed_date': 'N/A', 'state': 'Open', 'troubleshooting_conversation': [{'comment': '2023-09-21 09:30: @SupportMember2 sent email to customer and waiting for feedback from the customer about setting up a call to try and see what is going on.'}, {'comment': '2023-09-21 10:30: @SupportMember2 had call with the customer, there were some windows updates pending. They are going to do these updates after work today and feedback'}], 'resolution_notes': 'Not resolved'}
Thought:{
    "action": "Final Answer",
    "action_input": "@SupportMember2 is the support staff working with incident INC789. As per the troubleshooting conversation, they sent an email to t

In [13]:
answer = conversational_agent("""
    What is the current status of the incident and what are the next steps?
    """)
print(answer)



> Entering new AgentExecutor chain...
{
    "action": "Incident tool",
    "action_input": "INC789"
}
Observation: {'split_me': '|', 'incident_number': 'INC789', 'short_description': 'UI Application keeps crashing', 'long_description': 'Bob Jones at MegaCorpB has the app crashing when he clicks on detach. ', 'created_date': '2023-07-21', 'closed_date': 'N/A', 'state': 'Open', 'troubleshooting_conversation': [{'comment': '2023-09-21 09:30: @SupportMember2 sent email to customer and waiting for feedback from the customer about setting up a call to try and see what is going on.'}, {'comment': '2023-09-21 10:30: @SupportMember2 had call with the customer, there were some windows updates pending. They are going to do these updates after work today and feedback'}], 'resolution_notes': 'Not resolved'}
Thought:{
    "action": "Final Answer",
    "action_input": "The current status of the incident INC789 is still open. @SupportMember2 has been in contact with the customer. They sent an email 

In [14]:
format = """
Answer the following questions as best you can. You have access to the following tools:

python repl: useful for when you need to use python to answer a question. You should input python code
DuckDuckGo Search: Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input.
wikipedia: Useful for when you need to look up a topic, country or person on wikipedia

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python repl, DuckDuckGo Search, wikipedia]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""" 
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description", 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
)


In [17]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Incident tool: 
        Useful for looking up incidents, pass in an incident number to lookup an incident.        
        I an incident has a closed_date = 'N/A' then it's probaly still Open
        
Internal Knowledgebase tool: 
        Useful for querying about our internal services, teams and how to solve customer problems.
        The internal knowledge base has steps on how to solve customers problems, how our software systems work and which people and teams to reach out to. 
        Ask it for questions like 'What should I do?', 'What are the causes of this?', 'What do I do next?'
        

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Incident tool, Internal Knowledgebase tool]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/

In [21]:
# zero_shot_agent.run("Please help with INC123, what is it's status and can you assist with troubleshooting it?")
#zero_shot_agent.run("Please help with INC456, what is it's status and can you assist with troubleshooting it?")
zero_shot_agent.run("Please help with INC456, what is it's status and can you assist with troubleshooting it? What is the best team to help resolve this, please only give one team and how to contact them?")



> Entering new AgentExecutor chain...
 I need to find out the status of INC456 and the best team to contact for help.
Action: Incident tool
Action Input: INC456
Observation: {'split_me': '|', 'incident_number': 'INC456', 'short_description': 'User at CorpA not getting OTP', 'long_description': 'Mary Sue at CorpA is not getting on OTP when she tries to login', 'created_date': '2023-09-21', 'closed_date': 'N/A', 'state': 'Open', 'troubleshooting_conversation': [{'comment': '2023-09-21 13:00: @SupportMember1 reached out to the communication delivery team to ask what if there are issues, waiting on feedback.'}], 'resolution_notes': 'Still open'}
Thought: The incident is still open, so I need to find out which team to contact for help.
Action: Internal Knowledgebase tool
Action Input: What team should I contact for help with INC456?
Observation: [Document(page_content='Teams and People  \nFX Infrastructure Team  \nWhat can they help you w ith?  \nIssues with the Market Listener Service an

'The best team to contact for help with INC456 is the FX Infrastructure Team. You can contact them by sending an email to fxinfrateam@contoso.com.'